
#Evento evaluativo 2:

##Nombres: 
### Estiven Cano Urrego - Daniel Lopez - Kevin Julian Londoño 

Para el ejercicio del crucigrama resuelva:

1. Genere un CSP con las restricciones respectivas. Mínimamente debe tener restricciones sobre el cruce de palábras y el tamaño de las mismas. Por simplicidad se recomienda que cada variable corresponda a una palabra, no a una letra. Debe generar los operadores que considere necesarios para generar el CSP de forma correcta. Para que el ejercicio tenga sentido, debe agregar al dominio de cada una de las variables, además de la palabra correcta, varias palabras que puedan violar y no violar la restricción.

2. Solucionar el CSP por profundidad iterativa. Analizar el comportamiento del algoritmo. Para ello se recomienda activar los elementos de control con 
visualizar
. Concluya.

3. Solucionar el CSP por arcos consistentes y división de dominio. Analizar el comportamiento del algoritmo. Para ello se recomienda activar los elementos de control con 
visualizar
. Concluya.

4. Para el método de solución por arcos consistentes y división de dominio, genere un nuevo método llamado 
varias_soluciones
, que debe explorar más de una solución (pueden ser dos soluciones), si estas existen. Tenga en cuenta que una solución diferente puede darse si al menos una de las variables tiene una asignación diferente.

5. Repita el punto tres con el nuevo método. Concluya.

*No olvide comentar el código y agregar todas las funciones necesarias para que el notebook pueda funcionar de forma autónoma*

#Solución

##Librerias necesarias

### Clases para la busqueda

In [42]:
class Problema_busqueda():
    '''-nodo inicio
    - función de vecinos
    - meta (función booleana)
    - heurística'''

    def nodo_inicio(self):
        'retorna el nodo inicio'
        raise NotImplementedError('nodo_inicio')

    def es_meta(self, nodo):
        'retorna Verdadero si nodo es meta'
        raise NotImplementedError('es_meta')

    def heuristica(self, n):
        'retorna la heurística para el nodo n'
        return 0

    def vecinos(self, nodo):
        'retorna la lista de los arcos de los vecinos del nodo'
        raise NotImplementedError('vecinos')

In [43]:
class Arco():
    '''- nodo saliente
    - nodo entrante
    - costo (no negativo)'''

    def __init__(self, nodo_saliente, nodo_entrante, costo=1, accion=None):
        assert costo >= 0, ('El costo no puede ser negativo para'
                            + str(nodo_saliente) + ' -> ' + str(nodo_entrante))
        self.nodo_saliente = nodo_saliente
        self.nodo_entrante = nodo_entrante
        self.accion = accion
        self.costo = costo

    def __repr__(self):
        if self.accion:
            return str(self.nodo_saliente) + ' --' + str(self.accion) + ',' + str(self.costo) + '-> ' + str(
                self.nodo_entrante)
        else:
            return str(self.nodo_saliente) + ' --' + str(self.costo) + '-> ' + str(self.nodo_entrante)

In [44]:
class Camino():

    def __init__(self, inicial, arco=None):
        '''inicial puede ser un nodo o un camino'''
        self.inicial = inicial
        self.arco = arco
        if arco is None:
            self.costo = 0
        else:
            self.costo = inicial.costo + arco.costo

    def fin(self):
        '''retorna el nodo final del camino'''
        if self.arco is None:
            return self.inicial
        else:
            return self.arco.nodo_entrante  # retorna el nodo entrante del último arco

    def nodos(self):
        '''retorna los nodos del caimno de atrás hacia adelante'''
        actual = self
        while actual.arco is not None:
            yield actual.arco.nodo_entrante
            actual = actual.inicial
        yield actual.inicial

    def nodos_iniciales(self):
        '''retorna todos los nodos antes del nodo final'''
        if self.arco is not None:
            for nd in self.inicial.nodos(): yield nd

    def __repr__(self):
        if self.arco is None:
            return str(self.inicial)
        elif self.arco.accion:
            return (str(self.inicial) + "\n --" + str(self.arco.accion) + " --> " + str(self.arco.nodo_entrante))
        else:
            return (str(self.inicial) + "\n ----> " + str(self.arco.nodo_entrante))


In [45]:
class Visualizable():
    '''controla la cantidad de detalles por el nivel max_nivel_visual'''
    max_nivel_visual = 1

    def visualizar(self, nivel, *args, **nargs):
        if nivel <= self.max_nivel_visual:
            print(*args, **nargs)

In [46]:
class Buscador(Visualizable):
    '''retorna un buscador para un problema.
    Los caminos se pueden encontrar llamando repetidamente "buscar()"'''

    def __init__(self, problema):
        '''crea un buscador para el problema'''
        self.problema = problema
        self.inicializar_frontera()
        self.num_expansion = 0
        self.agregar_a_frontera(Camino(problema.nodo_inicio()))
        super().__init__()  # llama los atributos de la clase padre sin tener que conocer el nombre de la misma

    def inicializar_frontera(self):
        self.frontera = []

    def frontera_vacia(self):
        return self.frontera == []

    def agregar_a_frontera(self, camino):
        self.frontera.append(camino)

    # @visualizacion #decorador para modificar o replantear una función ya existente
    def buscar_profundidad(self):
        '''retorna el (siguiente) camino para el nodo inicio del problema
        al nodo meta. Retorna None si no existe el camino'''
        while not self.frontera_vacia():
            camino = self.frontera.pop()
            self.visualizar(3, "Expandiendo :", camino, "(costo: ", camino.costo, ")")
            self.num_expansion += 1
            if self.problema.es_meta(camino.fin()):  # se encontró la solución
                self.visualizar(3, self.num_expansion,
                                "caminos se han expandido y quedan ",
                                len(self.frontera), "caminos en la frontera")
                return camino
            else:
                vecis = self.problema.vecinos(camino.fin())
                self.visualizar(3, "Los vecinos son", vecis)
                for arco in reversed(list(vecis)):
                    self.agregar_a_frontera(Camino(camino, arco))
                self.visualizar(3, "Frontera", self.frontera)
            self.visualizar(3, "No hay más soluciones. Se expandieron un total de ",
                            self.num_expansion, "caminos")

    def buscar_anchura(self):
        '''retorna el (siguiente) camino para el nodo inicio del problema
        al nodo meta. Retorna None si no existe el camino'''
        while not self.frontera_vacia():
            camino = self.frontera.pop(0)
            self.visualizar(3, "Expandiendo :", camino, "(costo: ", camino.costo, ")")
            self.num_expansion += 1
            if self.problema.es_meta(camino.fin()):  # se encontró la solución
                self.visualizar(3, self.num_expansion,
                                "caminos se han expandido y quedan ",
                                len(self.frontera), "caminos en la frontera")
                return camino
            else:
                vecis = self.problema.vecinos(camino.fin())
                self.visualizar(3, "Los vecinos son", vecis)
                for arco in list(vecis):
                    self.agregar_a_frontera(Camino(camino, arco))
                self.visualizar(3, "Frontera", self.frontera)
            self.visualizar(3, "No hay más soluciones. Se expandieron un total de ",
                            self.num_expansion, "caminos")
            

##Clases y funciones para CSP

In [47]:
class Restriccion():
    '''Una restricción consiste de:
    - alcance: tupla o lista de variables
    - condición: una función que puede ser aplicada a una tupla de valores para las variables.
    Debe ser una función Booleana'''

    def __init__(self, alcance, condicion):
        self.alcance = alcance
        self.condicion = condicion

    def __repr__(self):
        return self.condicion.__name__  # + str(self.condicion)

    def holds(self, asignacion):
        '''retorna el valor de la restricción evaluada en la asignación'''
        return self.condicion(*tuple(asignacion[v] for v in self.alcance))

###Funciones usadas en las restricciones


In [48]:
'''Función que valida si la variable tiene la misma longitud que la pasada por parámetro'''
def numCaracteres(valor):
    noes_valor = lambda x: len(x) == valor
    return noes_valor

In [49]:
'''Función para asignar las posiciones en las que las variables deben encontrarse'''
def seEncuentran(valor1, valor2):
    noes_valor = lambda x, y: x[valor1] == y[valor2]
    return noes_valor

In [50]:
def union_diccionarios(d1,d2):
    '''retorna un diccionario que contiene las claves de d1 y d2.
    El valor de cada clave que está en d2 es el valor de d2, de otra forma
    es el valor de d1'''
    d = dict(d1) #copia d1
    d.update(d2)
    return d


In [51]:
class CSP:
    '''Un CSP requiere:
    - dominios : un diccionario que mapea las variables al conjunto de posibles valores.
    dominio[var] es el dominio de la variable var.
     - restricciones: conjunto o lista de objetos de clase Restricción'''

    def __init__(self, dominios, restricciones) -> object:
        self.variables = set(dominios)
        self.dominios = dominios
        self.restricciones = restricciones
        self.variables_a_restricciones = {var: set() for var in self.variables}
        for con in restricciones:
            for var in con.alcance:
                self.variables_a_restricciones[var].add(con)

    def __str__(self):
        '''representación del CSP'''
        return str(self.dominios)

    def __repr__(self):
        '''representación más detallada del CSP'''
        return "CSP( " + str(self.dominios) + " , " + str([str(c) for c in self.restricciones]) + " )"

    def consistencia(self, asignacion):
        '''asignación es variable: valor diccionario
        retorna Verdadero si todas las restricciones que pueden ser evaluadas, son evaluadas
        verdaderas dada una asignación'''
        return all(con.holds(asignacion) for con in self.restricciones if all(v in asignacion for v in con.alcance))



In [52]:
class Busqueda_CSP(Problema_busqueda):
    '''Un nodo es variable: valor diccionario'''

    def __init__(self, csp, orden_variable=None):
        self.csp = csp
        if orden_variable:
            assert set(orden_variable) == set(csp.variables)
            assert len(orden_variable) == len(csp.variables)
            self.variables = orden_variable
        else:
            self.variables = list(csp.variables)

    def es_meta(self, nodo):
        return len(nodo) == len(self.csp.variables)

    def nodo_inicio(self):
        return {}

    def vecinos(self, nodo):
        ''''este método usa el hecho de que la longitud del nodo, que es el número de variables
        asignadas, es el índice de la siguiente variable para hacer la división del grafo.
        Note que no se necesita revisar si hay más variables para hacer la división,
        dado que todos los nodos son consistentes por construcción y que no se tienen
        más variables  si hay una solución'''

        var = self.variables[len(nodo)]  # la siguiente variable
        res = []
        for val in self.csp.dominios[var]:
            nuevo = union_diccionarios(nodo, {var: val})
            if self.csp.consistencia(nuevo):
                res.append(Arco(nodo, nuevo))
        return res


Iniciamos una lista para las posibles soluciones

In [53]:
soluciones =[]

#Clase Solucionador_consistencia
### "Dentro se encuentran las funciones para una y varias soluciones"

In [54]:
class Solucionador_consistencia(Visualizable):
    '''Soluciona un CSP con consistencia de arco y división de dominio'''

    def __init__(self, csp, **kwargs):
        self.csp = csp
        super().__init__(**kwargs)

    def hacer_arcos_consistentes(self, dominios_originales=None, para_revisar=None):
        '''Hace que el CSP sea de arcos consistentes. dominios_originales son los dominios originales del CSP
        para_revisar es un conjunto de pares (variable, restriccion)'''

        if dominios_originales is None:
            dominios_originales = self.csp.dominios
        if para_revisar is None:
            para_revisar = {(variable, restriccion) for restriccion in self.csp.restricciones for variable in
                            restriccion.alcance}
        else:
            para_revisar = para_revisar.copy()
        dominios = dominios_originales.copy()
        self.visualizar(2, 'Realizando AC con dominios: ', dominios)
        while para_revisar:
            variable, restriccion = self.seleccionar_arco(para_revisar)
            self.visualizar(3, 'Procesando arco ( ', variable, ',', restriccion, ')')
            otras_variables = [otras for otras in restriccion.alcance if otras != variable]
            nuevo_dominio = {valor for valor in dominios[variable] if
                             self.se_mantiene(dominios, restriccion, {variable: valor},
                                              otras_variables)}
            if nuevo_dominio != dominios[variable]:
                self.visualizar(4, 'El arco : (', variable, ',', restriccion, ') es inconsistente')
                self.visualizar(3, 'Hay poda de dominio ', 'dominio(', variable, ')=', nuevo_dominio, ' debido a ',
                                restriccion)
                dominios[variable] = nuevo_dominio
                agregar_para_revisar = self.nuevo_para_revisar(variable, restriccion) - para_revisar
                para_revisar - agregar_para_revisar  # unión de conjuntos
                self.visualizar(3, ' agregando ', agregar_para_revisar if agregar_para_revisar else "nada",
                                ' para revisar')
            self.visualizar(4, "El arco (", variable, ',', restriccion, ") ahora es consistente")
        self.visualizar(2, 'Consistencia de arcos terminada. Dominios reducidos ', dominios)
        return dominios

    def nuevo_para_revisar(self, variable, restriccion):
        '''retorna nuevos elementos para agregar a para_revisar después de asignar variable en restriccion'''
        return {(nvariable, nrestriccion) for nrestriccion in self.csp.variables_a_restricciones[variable]
                if nrestriccion != restriccion
                for nvariable in nrestriccion.alcance
                if nvariable != variable}

    def seleccionar_arco(self, para_revisar):
        '''selecciona el arco del conjunto para_revisar, donde un arco es un par (variable, restriccion).
        El arco seleccionado debe ser removido del conjunto para_revisar'''
        return para_revisar.pop()

    def se_mantiene(self, dominios, restriccion, asignacion, otras_variables, ind=0):
        '''retorna Verdadero si restriccion se mantiene para una asignacion que se extiene con las variables en otras_variables[ind:]
        asignacion es un diccionario.'''
        if ind == len(otras_variables):
            return restriccion.holds(asignacion)
        else:
            variable = otras_variables[ind]
            for valor in dominios[variable]:
                asignacion[variable] = valor
                if self.se_mantiene(dominios, restriccion, asignacion, otras_variables, ind + 1):
                    return True
            return False

    def una_solucion(self, dominios=None, para_revisar=None):
        '''retorna una solución al CSP actual o retorna Falso si no hay soluciones. para_revisar es un conjunto de arcos que se deben revisar'''
        if dominios is None:
            dominios = self.csp.dominios
        nuevos_dominios = self.hacer_arcos_consistentes(dominios, para_revisar)
        if any(len(nuevos_dominios[variable]) == 0 for variable in
               dominios):  # alguna variable no tiene elementos en el dominio
            return False
        elif all(len(nuevos_dominios[variable]) == 1 for variable in dominios):  # ya se tiene una solución
            self.visualizar(2, "solución: ",
                            {variable: seleccionar(nuevos_dominios[variable]) for variable in nuevos_dominios})
            return {variable: seleccionar(nuevos_dominios[variable]) for variable in dominios}
        else:
            variable = self.seleccionar_variable(x for x in self.csp.variables if len(nuevos_dominios[x]) > 1)
            if variable:
                dominio1, dominio2 = particion_dominio(nuevos_dominios[variable])
                self.visualizar(2, "...particionando ", variable, " en ", dominio1, " y ", dominio2)
                nuevos_dominios1 = copiar_con_asignacion(nuevos_dominios, variable, dominio1)
                nuevos_dominios2 = copiar_con_asignacion(nuevos_dominios, variable, dominio2)
                para_revisar = self.nuevo_para_revisar(variable, None)
                self.visualizar(3, ' agregando ', para_revisar if para_revisar else " nada ", " para revisar ")
                return self.una_solucion(nuevos_dominios1, para_revisar) or self.una_solucion(nuevos_dominios2,
                                                                                              para_revisar)
    '''Función para encontrar las posibles soluciones de un CSP'''
    def varias_soluciones(self, dominios=None, para_revisar=None):
        '''Retorna las soluciones del CSP actual o retorna Falso si no hay soluciones.
        para_revisar es un conjunto de arcos que se deben resivarsa'''
        if dominios is None:
            dominios = self.csp.dominios
        nuevos_dominios = self.hacer_arcos_consistentes(dominios, para_revisar)
        if any(len(nuevos_dominios[variable]) == 0 for variable in
               dominios):  # Significa que alguna variable no tiene elementos en su dominio
            return False
        elif all(len(nuevos_dominios[variable]) == 1 for variable in dominios):
            # Se encontró una solución y se añade a la lista de soluciones
            soluciones.append({variable: seleccionar(nuevos_dominios[variable]) for variable in nuevos_dominios})
        else:
            variable = self.seleccionar_variable(x for x in self.csp.variables if len(nuevos_dominios[x]) > 1)
            if variable:
                dominio1, dominio2 = particion_dominio(nuevos_dominios[variable])
                self.visualizar(2, "...particionando ", variable, " en ", dominio1, " y ", dominio2)
                nuevos_dominios1 = copiar_con_asignacion(nuevos_dominios, variable, dominio1)
                nuevos_dominios2 = copiar_con_asignacion(nuevos_dominios, variable, dominio2)
                para_revisar = self.nuevo_para_revisar(variable, None)
                self.visualizar(3, ' agregando ', para_revisar if para_revisar else " nada ", " para revisar ")
                return self.varias_soluciones(nuevos_dominios1, para_revisar) or self.varias_soluciones(
                    nuevos_dominios2,
                    para_revisar)

    def seleccionar_variable(self, siguiente_variable):
        '''retorna la siguiente variable para hacer la división'''
        return seleccionar(siguiente_variable)


def particion_dominio(dominio):
    '''partición del dominio en dos'''
    dividir = len(dominio) // 2
    dominio1 = set(list(dominio)[:dividir])
    dominio2 = dominio - dominio1
    return dominio1, dominio2


def copiar_con_asignacion(dominios, variable=None, nuevo_dominio={True, False}):
    '''crea una copia de los dominios con asignación variable = nuevo_dominio. Si variable = None quiere decir que solo es una copia'''
    nuevosdominios = dominios.copy()
    if variable is not None:
        nuevosdominios[variable] = nuevo_dominio
    return nuevosdominios


def seleccionar(iterable):
    '''selecciona un elemento de iterable, retorna None si no existe dicho elemento.'''
    for e in iterable:
        return e


## Punto 1 - Creación del CSP

In [55]:
csp_crucigrama = CSP({'1': {'ITERATIVA', 'HISTORIAL', 'DELICIOSO', 'PELIGROSO'},
                      '2': {'VORAZ', 'SOBARA', 'ROBA', 'LABORAR'},
                      '3_h': {'ILUSTRATIVA', 'AESTRELLA', 'DELICIOSA', 'PELIGROSA'},
                      '3_v': {'ADVERSO', 'ANCHURA', 'ARTEMISA', 'EFECTOS'},
                      '4': {'ACOGIENDOSE', 'PROFUNDIDAD', 'COMUNIDADES', 'PROCRASTIAR'},
                      '5': {'TODO', 'ARCO', 'BARCO', 'OCRA'},
                      '6': {'COLINO', 'CAMINO', 'COMINO', 'CAMINOS'},
                      '7': {'NORTE', 'MONO', 'NODO', 'ODIN'}
                      },
                     [
                         Restriccion('1', numCaracteres(9)),
                         Restriccion('2', numCaracteres(5)),
                         Restriccion(('3_h',), numCaracteres(9)),
                         Restriccion(('3_v',), numCaracteres(7)),
                         Restriccion('4', numCaracteres(11)),
                         Restriccion('5', numCaracteres(4)),
                         Restriccion('6', numCaracteres(6)),
                         Restriccion('7', numCaracteres(4)),
                         Restriccion(('1', '3_h'), seEncuentran(1, 3)),
                         Restriccion(('3_v', '3_h'), seEncuentran(0, 0)),
                         Restriccion(('1', '4'), seEncuentran(3, 1)),
                         Restriccion(('1', '3_h'), seEncuentran(1, 3)),
                         Restriccion(('1', '6'), seEncuentran(8, 1)),
                         Restriccion(('6', '7'), seEncuentran(4, 0)),
                         Restriccion(('6', '5'), seEncuentran(5, 3)),
                         Restriccion(('4', '2'), seEncuentran(9, 3))]
                     )

##Punto 2 - Busqueda por profundidad iterativa

In [56]:
print(Buscador(Busqueda_CSP(csp_crucigrama)).buscar_profundidad())

{}
 ----> {'6': 'CAMINO'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA', '2': 'VORAZ'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA', '2': 'VORAZ', '4': 'PROFUNDIDAD'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA', '2': 'VORAZ', '4': 'PROFUNDIDAD', '7': 'NODO'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA', '2': 'VORAZ', '4': 'PROFUNDIDAD', '7': 'NODO', '3_h': 'AESTRELLA'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA', '2': 'VORAZ', '4': 'PROFUNDIDAD', '7': 'NODO', '3_h': 'AESTRELLA', '1': 'ITERATIVA'}
 ----> {'6': 'CAMINO', '3_v': 'ANCHURA', '2': 'VORAZ', '4': 'PROFUNDIDAD', '7': 'NODO', '3_h': 'AESTRELLA', '1': 'ITERATIVA', '5': 'TODO'}


##Punto 3 - Busqueda por arcos consistentes y división de dominio (Una solución)

In [57]:
print(Solucionador_consistencia(csp_crucigrama).una_solucion())

{'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ANCHURA', '4': 'PROFUNDIDAD', '5': 'TODO', '6': 'CAMINO', '7': 'NODO'}


##Punto 4 - Creación del método. 
La función se encuentra dentro de la clase 'Solucionador_consistencia' con el nombre de 'varias_soluciones'

##Punto 5 - Busqueda por arcos consistentes y división de dominio (Varias soluciones)

In [58]:
'''Se llama al metodo varias_soluciones para llenar la lista de las posibles soluciones'''
Solucionador_consistencia(csp_crucigrama).varias_soluciones()
cont = 0
for i in soluciones:
    cont = cont + 1
    print("Solución ", cont, ": ", i)


Solución  1 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ANCHURA', '4': 'PROFUNDIDAD', '5': 'TODO', '6': 'CAMINO', '7': 'NODO'}
Solución  2 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ANCHURA', '4': 'PROFUNDIDAD', '5': 'ARCO', '6': 'CAMINO', '7': 'NODO'}
Solución  3 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ANCHURA', '4': 'PROCRASTIAR', '5': 'TODO', '6': 'CAMINO', '7': 'NODO'}
Solución  4 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ANCHURA', '4': 'PROCRASTIAR', '5': 'ARCO', '6': 'CAMINO', '7': 'NODO'}
Solución  5 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ADVERSO', '4': 'PROFUNDIDAD', '5': 'TODO', '6': 'CAMINO', '7': 'NODO'}
Solución  6 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ADVERSO', '4': 'PROFUNDIDAD', '5': 'ARCO', '6': 'CAMINO', '7': 'NODO'}
Solución  7 :  {'1': 'ITERATIVA', '2': 'VORAZ', '3_h': 'AESTRELLA', '3_v': 'ADVERSO', '4': 'PROCRASTIAR', '5': 'TODO',